In [26]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
docu_txt = """주식회사 앨빈 - '슈퍼밴드' 10주년 기념 컴백 회의

2025년 3월 11일 (화) 14:00 - 16:00, 회사 회의실 A

참석자: 김앨빈 대표, 박다람 매니저, 이소리 홍보팀장, 최빛나 마케팅팀장, 강현우 프로듀서, 슈퍼밴드 멤버 전원

회의 시작. 김앨빈 대표 "이제 10주년인데 뭔가 크게 가야 하지 않겠어?" 다들 동의

일정 조율
발매일 6월 15일 확정. 멤버들 일정 문제 없음
티저 5월 1일부터 하나씩 공개. SNS 활용
뮤비 촬영 4월 10~12일. 로케이션 검토 중
쇼케이스 6월 14일로 가자. 장소는 아직 미정, 빠른 섭외 필요
월드투어? 일단 논의만, 일정 잡긴 어려움

앨범 컨셉 논의
앨범명 아이디어: Reignite, Revival, Firestorm (Reignite로 가는 분위기)
컨셉: 10년을 돌아보고 다시 불태우는 느낌. 레트로+현대 락 스타일로 가자
타이틀곡 가제: 불꽃처럼. 강렬한 락 사운드, 희망적인 가사
수록곡 10곡 예정. 기존 곡 리메이크도 넣을지 고민

마케팅 전략
SNS 티저 영상, 멤버 개인 브이로그 촬영 진행
틱톡 챌린지 같은 거 해야 하나? 다들 긍정적 반응
팬미팅, 사인회 6월 중에. 세부 일정은 조율 필요
미디어 인터뷰 잡아야 함. 음악방송도 적극 참여
패션 브랜드 콜라보 제안 있음. 검토 필요

기타 논의
굿즈 제작? 팬클럽 한정 프리오더 진행 고려
다큐멘터리 제작할까? 가능성 있음, 제작팀 컨택 예정
멤버들 건강 스케줄 조정 신경 써야 함

결론
3월 말까지 세부 일정 확정
다음 회의 3월 25일 14:00

회의 종료 16:00"""

In [2]:
email_txt = """
제목: [마케팅팀 회의 공지] '슈퍼밴드' 10주년 컴백 마케팅 전략 논의

팀원 여러분,

안녕하세요. 최빛나 팀장입니다.
오늘 진행된 '슈퍼밴드' 10주년 컴백 회의 내용을 공유드립니다.

컴백 일정이 6월 15일로 확정되었고, 이에 맞춰 마케팅 전략을 구체화해야 합니다. 주요 논의 사항은 다음과 같습니다.

SNS 캠페인: 티저 영상 및 멤버별 브이로그 콘텐츠 기획
틱톡 챌린지: 팬 참여 유도 방안 논의
오프라인 이벤트: 팬미팅, 사인회 일정 및 진행 방식 검토
미디어 홍보: 음악방송 및 인터뷰 스케줄 조율
콜라보레이션: 패션 브랜드 협업 가능성 검토

이에 따라 마케팅팀 내부 회의를 진행하려 합니다.
📅 일정: 3월 13일(수) 오후 2시
📍 장소: 마케팅팀 회의실

각자 아이디어를 정리해 오시면 원활한 논의가 가능할 것 같습니다.
질문이나 제안이 있으면 미리 공유해주세요.

감사합니다.
최빛나 드림
"""

### 요약봇(Agent)
- 회의록 요약 기능과 이메일 요약 기능을 함수로 작성 > Tools로 생성
    - 서로 다른 프롬프트로 LLM 질의 > 응답 반환
    - langchain.tools 패키지의 tool 활용
- agent를 만들어 요청을 처리할 수 있도록 할 것

In [17]:
# LLM
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

# 회의록 요약 기능 함수
def summarize_doc(doc_txt):
    
    prompt = f"""
    다음 조건에 맞게 회의 내용을 요약해줘.
    
    <조건>
    1. 회의 장소, 일자, 참여자는 각각 작성할것
    2. 회의 내용을 대주제, 소주제로 나누어 작성할 것
    3. 요약한 회의 내용은 불렛 형식으로 작성할 것
    
    <요약할 회의 내용>
    {doc_txt}
    """
    return model.invoke(prompt).content


# 이메일 요약 기능 함수
def summarize_email(email_txt):
    prompt = f"""
    다음 조건에 맞게 이메일 내용을 요약해줘.
    
    <조건>
    1. 이메일 발신자를 따로 표시할것 
    2. 메일 내용을 간략하게 요약해서 한 줄로 표현할 것
    3. 요청받은 업무 사항이 있으면 불렛 형식으로 표현할 것
    
    <요약할 이메일 내용>
    {email_txt}
    """
    return model.invoke(prompt).content


# Tool 생성 > Agent 생성 > 테스트 데이터로 확인
from langchain.tools import Tool
from langchain.agents import AgentType, initialize_agent, load_tools

tools = [
    Tool(name='회의 요약', func=summarize_doc, description='긴 회의록 내용 요약'),
    Tool(name='이메일 요약', func=summarize_email, description='업무 이메일 내용 요약')
]
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_error=True,
    verbose=True
)

In [18]:
agent.run('회의 내용 요약:' + summarize_doc(docu_txt))

C:\Users\ljh10\AppData\Local\Temp\ipykernel_7548\78859743.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run('회의 내용 요약:' + summarize_doc(docu_txt))




> Entering new AgentExecutor chain...
I need to summarize the provided meeting notes regarding the 10th anniversary comeback plan for 'Superband'. 

Action: 회의 요약  
Action Input: ### 회의 요약

**회의 장소:** 회사 회의실 A  
**일자:** 2025년 3월 11일 (화) 14:00 - 16:00  
**참여자:** 김앨빈 대표, 박다람 매니저, 이소리 홍보팀장, 최빛나 마케팅팀장, 강현우 프로듀서, 슈퍼밴드 멤버 전원  

---

#### 대주제: '슈퍼밴드' 10주년 기념 컴백 계획

- **일정 조율**
  - 발매일: 6월 15일 확정, 멤버들 일정 문제 없음.
  - 티저 공개: 5월 1일부터 SNS 활용하여 하나씩 공개.
  - 뮤비 촬영: 4월 10~12일, 로케이션 검토 중.
  - 쇼케이스: 6월 14일, 장소는 미정, 빠른 섭외 필요.
  - 월드투어: 논의 중이나 일정 잡기 어려움.

- **앨범 컨셉 논의**
  - 앨범명 아이디어: Reignite, Revival, Firestorm (Reignite로 가는 분위기).
  - 컨셉: 10년을 돌아보고 다시 불태우는 느낌, 레트로+현대 락 스타일.
  - 타이틀곡 가제: 불꽃처럼, 강렬한 락 사운드와 희망적인 가사.
  - 수록곡: 10곡 예정, 기존 곡 리메이크 여부 고민.

- **마케팅 전략**
  - SNS 티저 영상 및 멤버 개인 브이로그 촬영 진행.
  - 틱톡 챌린지 진행 여부 긍정적 반응.
  - 팬미팅 및 사인회: 6월 중, 세부 일정 조율 필요.
  - 미디어 인터뷰 및 음악방송 적극 참여.
  - 패션 브랜드 콜라보 제안 검토 필요.

- **기타 논의**
  - 굿즈 제작: 팬클럽 한정 프리오더 진행 고려.
  - 다큐멘터리 제작 가능성 있음, 제작팀 컨택 예정.
  - 멤버들 건강 및 스케줄 조정 신경 써야 함.


"The meeting focused on the 10th anniversary comeback plan for 'Superband', confirming the release date for June 15, teaser releases starting May 1, and discussing album concepts and marketing strategies. A follow-up meeting is scheduled for March 25."

In [19]:
agent.run('이메일 내용 요약:' + summarize_email(email_txt))



> Entering new AgentExecutor chain...
I need to summarize the content of the email regarding the marketing strategy meeting for the 10th anniversary comeback of 'Superband'. 

Action: 이메일 요약  
Action Input: **발신자:** 최빛나 팀장  
**요약:** '슈퍼밴드' 10주년 컴백을 위한 마케팅 전략 회의 공지.  
**요청 사항:**  
- SNS 캠페인: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- 틱톡 챌린지: 팬 참여 유도 방안 논의  
- 오프라인 이벤트: 팬미팅, 사인회 일정 및 진행 방식 검토  
- 미디어 홍보: 음악방송 및 인터뷰 스케줄 조율  
- 콜라보레이션: 패션 브랜드 협업 가능성 검토  

Observation: **발신자:** 최빛나 팀장  
**요약:** '슈퍼밴드' 10주년 컴백을 위한 마케팅 전략 회의 공지.  
**요청 사항:**  
- SNS 캠페인: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- 틱톡 챌린지: 팬 참여 유도 방안 논의  
- 오프라인 이벤트: 팬미팅, 사인회 일정 및 진행 방식 검토  
- 미디어 홍보: 음악방송 및 인터뷰 스케줄 조율  
- 콜라보레이션: 패션 브랜드 협업 가능성 검토  
Thought:I have successfully summarized the email regarding the marketing strategy meeting for 'Superband's' 10th anniversary comeback. 

Final Answer: **발신자:** 최빛나 팀장  
**요약:** '슈퍼밴드' 10주년 컴백을 위한 마케팅 전략 회의 공지.  
**요청 사항:**  
- SNS 캠페인: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  
- 틱톡 챌린지: 팬 참여 유도 방안 논의  
- 오프라인 이벤트: 팬미팅, 사인회 일정

"**발신자:** 최빛나 팀장  \n**요약:** '슈퍼밴드' 10주년 컴백을 위한 마케팅 전략 회의 공지.  \n**요청 사항:**  \n- SNS 캠페인: 티저 영상 및 멤버별 브이로그 콘텐츠 기획  \n- 틱톡 챌린지: 팬 참여 유도 방안 논의  \n- 오프라인 이벤트: 팬미팅, 사인회 일정 및 진행 방식 검토  \n- 미디어 홍보: 음악방송 및 인터뷰 스케줄 조율  \n- 콜라보레이션: 패션 브랜드 협업 가능성 검토"

### News 매니저

In [11]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32142 sha256=63f6ce725903143b398c5dcffc064e24e6b39daa0603303b2d9aff3df1d00493
  Stored in directory: c:\users\ljh10\appdata\local\pip\cache\wheels\68\8e\73\744b7d9d7ac618849d93081a20e1c0deccd2aef90901c9f5a9
Successfully built google-search-results


### News 매니저
1. 최신 뉴스 검색
    - langchain.utilities 패키지의 SerpAPIWrapper
2. 뉴스 요약 및 키워드 분석
3. 사용자 맞춤형 뉴스 추천

In [50]:
from langchain.utilities import SerpAPIWrapper
param= {
    'engine': 'google_news',
    'gl' : 'kr',
    'hl' : 'ko'
}
serp = SerpAPIWrapper(params=param)
# news_text = serp.run('코로나')
# news_text[:2]

In [51]:
from langchain.utilities import SerpAPIWrapper

news_txt = input('뉴스 키워드를 입력하세요:')
news_text =serp.run(news_txt)
news = news_text[:2]
model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

def summary_news(news_text):
    prompt = f"""
    다음 뉴스 내용을 요약해줘.
    
    <조건>
    최신 뉴스 3개만 정리할 것
    키워드와 관련된 뉴스만 정리할 것
    각 뉴스별로 요약하고 핵심 키워드 분석할 것
    다른 연관된 뉴스 추천해줄 것
    
    <요약할 뉴스 내용>
    {news_text}
    """
    
    return model.invoke(prompt).content

tools = [Tool(
    name='뉴스 요약',
    func=summary_news,
    description='뉴스 요약'
)]

agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_error=True,
    verbose=True
)

agent.run('뉴스 요약:' + summary_news(news))



> Entering new AgentExecutor chain...
Question: 뉴스 요약:### 뉴스 요약

Thought: I need to summarize the news articles provided. 
Action: 뉴스 요약
Action Input: "외국인 관광객 수가 코로나19 이전 수준을 초과하며, 관광업계의 회복세가 두드러지고 있다. 이는 계엄 한파에도 불구하고 나타난 현상으로, 관광 산업의 재개가 긍정적인 신호로 해석되고 있다. 최근 두 달 간 자영업자 20만명이 폐업한 것으로 나타났다. 많은 자영업자들이 코로나19 이후 회복을 기대했으나, 실제로는 더 어려운 상황에 직면하고 있음을 보여준다. 자영업자들이 코로나19 시기보다 더 힘든 상황에 처해 있으며, 최근 두 달 동안 20만명 이상이 폐업했다는 보고가 이어지고 있다. 이는 경제 회복의 지연을 반영한다."
Observation: ### 뉴스 요약 및 키워드 분석

1. **뉴스 제목: 외국인 관광객 수 증가**
   - **요약**: 외국인 관광객 수가 코로나19 이전 수준을 초과하며, 관광업계의 회복세가 두드러지고 있다. 이는 계엄 한파에도 불구하고 관광 산업의 재개가 긍정적인 신호로 해석되고 있다.
   - **핵심 키워드**: 외국인 관광객, 코로나19, 관광업계 회복, 긍정적 신호

2. **뉴스 제목: 자영업자 폐업 증가**
   - **요약**: 최근 두 달 간 자영업자 20만명이 폐업한 것으로 나타났다. 많은 자영업자들이 코로나19 이후 회복을 기대했으나, 실제로는 더 어려운 상황에 직면하고 있음을 보여준다.
   - **핵심 키워드**: 자영업자, 폐업, 코로나19, 경제 회복 지연

3. **뉴스 제목: 자영업자들의 어려운 상황**
   - **요약**: 자영업자들이 코로나19 시기보다 더 힘든 상황에 처해 있으며, 최근 두 달 동안 20만명 이상이 폐업했다는 보고가 이어지고 있다. 이는 경제 회복의 지연을 반영한다.
   - **핵심 키워드**: 자영업자, 

'외국인 관광객 수가 코로나19 이전 수준을 초과하며 관광업계가 회복세를 보이고 있지만, 자영업자들은 최근 두 달 동안 20만명이 폐업하는 등 더 어려운 상황에 처해 있으며, 이는 경제 회복의 지연을 반영하고 있다.'